In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {
              'price':'{:.2f}','unit_cost':'{:.2f}','qty':'{:,}',
              'amt':'{:,.2f}','gross':'{:,.2f}','net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}'
              }

### Average cost calculation

In [2]:
name = 'IMH'

In [3]:
sql = '''
SELECT B.id, S.name, date, qty, price, qty*price AS gross, fee, vat, net, status, kind
FROM buys B
JOIN stocks S ON B.stock_id = S.id 
WHERE name = "%s" AND status = "Active"
ORDER BY id DESC'''
sql = sql % name
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

,id,name,date,qty,price,gross,fee,vat,net,status,kind
0,1108,IMH,2022-01-26,"6,000",17.00,"102,000.00",211.14,14.78,"102,225.92",Active,DTD
1,1096,IMH,2022-01-11,"7,000",20.00,"140,000.00",289.80,20.29,"140,310.09",Active,DTD
2,1069,IMH,2021-12-08,"2,000",11.00,"22,000.00",45.54,3.19,"22,048.73",Active,DTD


In [4]:
buys_grp = buys.groupby('name')[['gross','qty']].sum()
buys_grp['unit_cost'] = buys_grp.gross / buys_grp.qty
buys_grp.style.format(format_dict)

,gross,qty,unit_cost
name,,,
IMH,"264,000.00","15,000",17.60


### Recalculate net cost by id

In [5]:
ids = buys['id']
ids

0    1108
1    1096
2    1069
Name: id, dtype: int64

In [6]:
new_price = 17.60

In [7]:
def update_buys_by_id(new_price,id):
    sql = "UPDATE buys SET price = %s, fee = qty*price*.00207, vat = fee*.07, net = (qty*price)+fee+vat WHERE id = %s"
    sql = sql % (new_price,id)
    rp = conpf.execute(sql)
    return "Records updated = " + str(id)

In [8]:
sql = '''
SELECT B.id, S.name, date, qty, price, qty*price AS gross, fee, vat, net, status, kind
FROM buys B
JOIN stocks S ON B.stock_id = S.id 
WHERE name = "%s" AND status = "Active"
ORDER BY id DESC'''
sql = sql % name
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

,id,name,date,qty,price,gross,fee,vat,net,status,kind
0,1108,IMH,2022-01-26,"6,000",17.00,"102,000.00",211.14,14.78,"102,225.92",Active,DTD
1,1096,IMH,2022-01-11,"7,000",20.00,"140,000.00",289.80,20.29,"140,310.09",Active,DTD
2,1069,IMH,2021-12-08,"2,000",11.00,"22,000.00",45.54,3.19,"22,048.73",Active,DTD


In [9]:
i = 0
for id in ids:
    update_buys_by_id(new_price,id)
    i += 1

In [10]:
sql = '''
SELECT B.id, S.name, date, qty, price, qty*price AS gross, fee, vat, net, status, kind
FROM buys B
JOIN stocks S ON B.stock_id = S.id 
WHERE name = "%s" AND status = "Active"
ORDER BY id DESC'''
sql = sql % name
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

,id,name,date,qty,price,gross,fee,vat,net,status,kind
0,1108,IMH,2022-01-26,"6,000",17.60,"105,600.00",218.59,15.30,"105,833.89",Active,DTD
1,1096,IMH,2022-01-11,"7,000",17.60,"123,200.00",255.02,17.85,"123,472.87",Active,DTD
2,1069,IMH,2021-12-08,"2,000",17.60,"35,200.00",72.86,5.10,"35,277.96",Active,DTD
